# OGGM glacier simulator

App to initialize a simple glacier and look how it develops under changing mass balance profiles.

## Import packages

import plotting libaries

In [ ]:
import holoviews as hv
from holoviews import opts
from holoviews.streams import Stream, param
import panel as pn

# is needed for linecolor by velocity/thickness colorplots
from bokeh.transform import linear_cmap

hv.extension('bokeh', width=100)
pn.extension()

import OGGM packages

In [ ]:
# Constants for initialization
from functools import partial
from oggm.core.flowline import FluxBasedModel, RectangularBedFlowline, TrapezoidalBedFlowline, ParabolicBedFlowline
from oggm.core.massbalance import LinearMassBalance
from oggm import cfg
cfg.initialize()

# There are several solvers in OGGM core. We use the default one for this experiment
FlowlineModel = partial(FluxBasedModel, min_dt=0, cfl_number=0.01)

and some other useful packages

In [ ]:
import numpy as np

## Define some variables for model and plotting

In [ ]:
# number of steps from bottem to top of glacier
nx = 200

# model grid spacing in m
map_dx = 100

# distance along glacier (x-axis of glacier profil plot) in km
distance_along_glacier = np.linspace(0, nx, nx) * map_dx * 1e-3

# default tools for plot
default_tools = ['save', 'box_zoom', 'reset']

# glacier top height
glacier_top_height = 4000

# glacier bottom height
glacier_bottom_height = 0

# Variable to follow which text is currently displayed in the headerline
header_text_status = 0

# colormap for velocity and thickness
colormap = ['#5ebaff', '#00faf4', '#ffffcc',
            '#ffe775', '#ffc140', '#ff8f20', '#ff6060']

# define nan array for glacier outline plotting in width plot
nan_array = np.empty(nx)
nan_array.fill(np.nan)

## Define values for slider:

In [ ]:
# ELA height
ELA_height_min = 0
ELA_height_max = 4000
ELA_height_default = 3000
ELA_height_step = 100

# mass balance gradient
mb_gradient_max = 15
mb_gradient_min = 1
mb_gradient_default = 4
mb_gradient_step = 1

# years of calculation
years_min = 10
years_max = 1000
years_default = 150
years_step = 10

# linear bed rock slope
bed_rock_slope_values = ['steepest', 'steeper',
                         'steep', 'flat', 'flatter', 'flattest']
bed_rock_slope_default = 'flat'

# bed rock profile
bed_rock_profile_values = [
    'linear', 'getting flatter', 'getting steeper', 'cliff']
bed_rock_profile_default = 'linear'

# bed rock width
bed_rock_width_values = ['constant', 'getting narrower', 'getting wider',
                         'wide at the top and narrow at the bottom']
bed_rock_width_default = 'constant'

# Glen's creep parameter
glens_creep_parameter_values = ['small', 'default', 'large']
glens_creep_parameter_default = 'default'

# sliding parameter
sliding_parameter_values = ['no sliding', 'sliding']
sliding_parameter_default = 'no sliding'

# model dyears (how often the model state should be shown)
dyears_model_start = 1
dyears_model_end = 100
dyears_model_step = 1
dyears_model_default = 10

# maximum calculation years of model (before it gets interrupted)
max_calc_years_start = 100
max_calc_years_end = 5000
max_calc_years_step = 100
max_calc_years_default = 1000

## Define global model variables:

In [ ]:
bed_h = np.linspace(glacier_top_height, 0, nx)
mb_model = []
model = []
run_surface_heights = []
glacier_outlines = []
glen_a = cfg.PARAMS['glen_a']
fs = 0
ELA_x_position = 0

# define width, so glacier bed could be created when plot is created (after creation model is ready)
widths = np.zeros(nx) + 4.

# is needed so there is always a 'change' and the dynamic plot is reacting
run_loop_count = 0

# years after the model progress should be shown
dyears = dyears_model_default

# years after model should be aborted
max_calc_years = max_calc_years_default

## Define global curves:

In [ ]:
bed_rock_height_curve = []
bed_rock_width_curve = []
glacier_height_curve = []
mb_curve = []
width_curve = []
info_text = []

## Define Headerline with help functionality

### help functions for Headerline buttons

In [ ]:
def next_help_button_click(arg=None):
    global header_text_status

    header_text_status += 1

    help_function()


def previous_help_button_click(arg=None):
    global header_text_status

    header_text_status -= 1

    help_function()

### define elements of Headerline

In [ ]:
header_text_default = ('<div style="font-size:20px">OGGM edu glacier simulator</div>' +
                       'With this app you can simulate idealized glaciers with the ' +
                       '<a href="https://oggm.org">OGGM</a> model. If you are not familiar with glaciers at all ' +
                       'click on "next" to go through an introduction. <br>This app was realised ' +
                       'using tools of <a href="https://holoviz.org">HoloViz</a>.')
header_text = pn.pane.Markdown(
    header_text_default, sizing_mode='stretch_width')

oggm_edu_logo = '<a href="http://edu.oggm.org"><img src="http://edu.oggm.org/en/latest/_images/oggm.gif" width=170></a>'
hv_logo = '<a href="https://holoviz.org"><img src="https://holoviz.org/assets/holoviz-logo-stacked.svg" width=80></a>'

next_help_button = pn.widgets.Button(disabled=False,
                                     name='next',
                                     button_type='success',
                                     width=75)
next_help_button.param.watch(next_help_button_click, 'clicks')

previous_help_button = pn.widgets.Button(disabled=True,
                                         name='previous',
                                         button_type='danger',
                                         width=75)
previous_help_button.param.watch(previous_help_button_click, 'clicks');

### Put actual Headerline together

In [ ]:
header = pn.Row(pn.Pane(oggm_edu_logo),
                pn.layout.Spacer(width=10),
                header_text,
                pn.Column(pn.layout.Spacer(),
                          previous_help_button,
                          next_help_button),
                pn.layout.Spacer(),
                pn.Pane(hv_logo, width=80), height=120, sizing_mode='stretch_width')

### define actual help function with help text

In [ ]:
def help_function():
    global header_text_status

    if header_text_status == 0:
        header_text.object = header_text_default
        previous_help_button.disabled = True

    elif header_text_status == 1:
        header_text.object = ('<div style="font-size:20px">OGGM edu glacier simulator beginner tour</div>' +
                              'Welcome to the beginner tour of the <a href="http://edu.oggm.org">OGGM edu</a> ' +
                              'glacier simulater. To navigate through the tour use the "next" button to the right ' +
                              'to continue the tour or "previous" to see the last page again. So lets start with a ' +
                              'click on "next"!')
        previous_help_button.disabled = False

    elif header_text_status == 2:
        tab_menu.active = 0
        header_text.object = ('First let\'s have a look at the figures on the left side. The top figure shows ' +
                              'the glacierbed from the side in gray. The dark blue line shows the current glacier ' +
                              'state and you can see at the beginning this line follows the glacierbed, so there ' +
                              'is no glacier. The figure below shows the glacier from above and the dark gray areas ' +
                              'indicate the boarder of the glacierbed. So now click on the "run the model" ' +
                              'button to see the glacier grow and then on "next".')

    elif header_text_status == 3:
        header_text.object = ('So now you can see the thickness of the glacier in the top figure and the length of ' +
                              'the glacier in the bottom figure. If you want to zoom in you can do this by clicking ' +
                              'into one of the figures, hold the click and draw an rectangle. You can see that if ' +
                              'you zoom in into one figure the others automatically rezoom to the same extent. ' +
                              'To go back to the original extent click on the "Reset" button to the right of the ' +
                              'figures (two arrows forming a circle). After playing around with the zoom click ' +
                              'on "next".')

    elif header_text_status == 4:
        header_text.object = ('Now we are concentrating on the figure in the top right corner. This figure shows ' +
                              'the height dependent annual mass balance. But what is the annual mass balance? ' +
                              'The unit is meter per year, so when the annual mass balance at a specific height ' +
                              'is positive the glacier had gain mass during one year, so in other words ' +
                              'there were more accumulation (mass gain through e.g. snowfall or wind drift) ' +
                              'than ablation (mass loss through e.g. melting or wind drift) happening. Such a positive ' +
                              'annual mass balance is indicated with the green area in the figure and red indicates ' +
                              'a mass loss. ("next")')

    elif header_text_status == 5:
        header_text.object = ('In between the green (mass gain) and the red area (mass loss) you can see a dashed ' +
                              'black line with the annotation ELA, which stands for Equilibrium Line Altitude. ' +
                              'At this altitude the glacier do not change his mass during one year. And the black ' +
                              'solid line shows the actual mass balance for each altitude band. Through the glacier ' +
                              'flow mass is transported from higher altitudes down. For a more detailed ' +
                              'explenation of the mass balance concept you can click on this ' +
                              '<a href=http://www.antarcticglaciers.org/glacier-processes/introduction-glacier-mass-' +
                              'balance/#SECTION_1>link</a>. ("next")')

    elif header_text_status == 6:
        header_text.object = ('Summarizing the annual mass balance determines the ' +
                              '"behavior" of the glacier, e.g. how large can it become, at which altitudes the glacier ' +
                              'can survive. So always you click on the "run the model" button the model is starting ' +
                              'from zero and develops the glacier until it is in equilibrium with annual mass balance, ' +
                              'this means the glacier geometry is not changing anymore, although internal the glacier ' +
                              'is still moving mass from higher altitudes to lower ones. So now we start playing ' +
                              'around with the model and look how different parameters change the geometry of the ' +
                              'glacier. ("next")')
        next_help_button.disabled = False

    elif header_text_status == 7:
        header_text.object = ('You can see that there are three parameters to play with in the beginners mode: the ' +
                              'slope of the bedrock, the width of the bedrock and the ELA (equilibrium line altitude). ' +
                              'At the beginning you should only change one parameter to see how it effects the glacier.' +
                              '(end of Tour until now, more will come soon)')
        next_help_button.disabled = True

## Define Panels for tab menu

In [ ]:
# define background color of tab menu
tab_background = '#fafafa'

### Panel for beginner mode

beginner button function

In [ ]:
def beginner_button_click(arg=None):
    # set Variables not changeable in beginner mode to there default values
    mb_gradient.value = mb_gradient_default
    bed_rock_profile.value = bed_rock_profile_default
    glens_creep_parameter.value = glens_creep_parameter_default
    sliding_parameter.value = sliding_parameter_default

    # for consitency set values of the advanced mode
    ELA_height_advanced.value = ELA_height.value
    bed_rock_width_advanced.value = bed_rock_width.value
    bed_rock_slope_advanced.value = bed_rock_slope.value

    # set numeric values out of selection
    set_glen_a()
    set_fs()

    run_the_model(stage='beginner mode')

actual beginner Panel

In [ ]:
bed_rock_slope = pn.widgets.Select(name='slope of bedrock profile',
                                   options=bed_rock_slope_values,
                                   value=bed_rock_slope_default,
                                   sizing_mode='stretch_width')

bed_rock_width = pn.widgets.Select(name='width along glacier',
                                   options=bed_rock_width_values,
                                   value=bed_rock_width_default,
                                   sizing_mode='stretch_width')

ELA_height = pn.widgets.IntSlider(name='ELA (equilibrium line altitude)',
                                  start=ELA_height_min,
                                  end=ELA_height_max,
                                  step=ELA_height_step,
                                  value=ELA_height_default,
                                  sizing_mode='stretch_width')

beginner_button = pn.widgets.Button(name='run the model',
                                    button_type='primary')
beginner_button.param.watch(beginner_button_click, 'clicks')

beginner_panel = pn.Column(pn.Row(bed_rock_slope, bed_rock_width, ELA_height),
                           beginner_button,
                           background=tab_background,
                           sizing_mode='stretch_width')

### Panel for advanced mode

advanced button help functions

In [ ]:
def advance_model_click():
    # for consitency set values of the advanced mode
    ELA_height.value = ELA_height_advanced.value
    bed_rock_width.value = bed_rock_width_advanced.value
    bed_rock_slope.value = bed_rock_slope_advanced.value

    # set numeric values out of selection
    set_glen_a()
    set_fs()

advanced button functions

In [ ]:
def run_model_equilibrium_button_click(arg=None):
    advance_model_click()
    run_the_model(stage='advance equilibrium')


def run_model_years_button_click(arg=None):
    advance_model_click()
    run_the_model(stage='advance years')


def create_new_model_button_click(arg=None):
    bed_rock_width.value = bed_rock_width_advanced.value
    bed_rock_slope.value = bed_rock_slope_advanced.value

    main_figure.event(button_name='new model',
                      clicks=create_new_model_button.clicks)

actual advanced Panel

In [ ]:
mb_gradient = pn.widgets.IntSlider(name='mass balance gradient',
                                   start=mb_gradient_min,
                                   end=mb_gradient_max,
                                   step=mb_gradient_step,
                                   value=mb_gradient_default,
                                   sizing_mode='stretch_width')

bed_rock_profile = pn.widgets.Select(name='bedrock profile',
                                     options=bed_rock_profile_values,
                                     value=bed_rock_profile_default,
                                     sizing_mode='stretch_width')

ELA_height_advanced = pn.widgets.IntSlider(name='ELA',
                                           start=ELA_height_min,
                                           end=ELA_height_max,
                                           step=ELA_height_step,
                                           value=ELA_height_default,
                                           sizing_mode='stretch_width')

bed_rock_slope_advanced = pn.widgets.Select(name='slope of linear bedrock profile',
                                            options=bed_rock_slope_values,
                                            value=bed_rock_slope_default,
                                            sizing_mode='stretch_width')

bed_rock_width_advanced = pn.widgets.Select(name='width along glacier',
                                            options=bed_rock_width_values,
                                            value=bed_rock_width_default,
                                            sizing_mode='stretch_width')

glens_creep_parameter = pn.widgets.Select(name="Glen's creep parameter",
                                          options=glens_creep_parameter_values,
                                          value=glens_creep_parameter_default,
                                          sizing_mode='stretch_width')

sliding_parameter = pn.widgets.RadioBoxGroup(name='Sliding parameter',
                                             options=sliding_parameter_values,
                                             value=sliding_parameter_default,
                                             inline=True,
                                             sizing_mode='stretch_width')

run_model_years = pn.widgets.IntSlider(name='years to advance model',
                                       start=years_min,
                                       end=years_max,
                                       step=years_step,
                                       value=years_default,
                                       sizing_mode='stretch_width')

run_model_equilibrium_button = pn.widgets.Button(name='run to equilibrium',
                                                 button_type='primary',
                                                 sizing_mode='stretch_width')
run_model_equilibrium_button.param.watch(
    run_model_equilibrium_button_click, 'clicks')

run_model_years_button = pn.widgets.Button(name='advance model',
                                           button_type='primary',
                                           sizing_mode='stretch_width')
run_model_years_button.param.watch(run_model_years_button_click, 'clicks')

create_new_model_button = pn.widgets.Button(name='start new model',
                                            button_type='primary',
                                            sizing_mode='stretch_width')
create_new_model_button.param.watch(create_new_model_button_click, 'clicks')

advanced_panel = pn.Tabs(('run the model',
                          pn.Row(pn.Column(ELA_height_advanced,
                                           mb_gradient,
                                           sizing_mode='stretch_width'),
                                 pn.Column(glens_creep_parameter,
                                           sliding_parameter,
                                           sizing_mode='stretch_width'),
                                 pn.Column(run_model_years,
                                           sizing_mode='stretch_width'),
                                 pn.Column(run_model_years_button,
                                           run_model_equilibrium_button,
                                           sizing_mode='stretch_width'),
                                 sizing_mode='stretch_width')),
                         ('create new model',
                          pn.Column(pn.Row(bed_rock_profile,
                                           bed_rock_slope_advanced,
                                           bed_rock_width_advanced),
                                    create_new_model_button,
                                    sizing_mode='stretch_width')),
                         background=tab_background)

### Panel for help

help functions for buttons

In [ ]:
def beginner_introduction_button_click(arg=None):
    global header_text_status

    # just to be sure the buttons are activated
    next_help_button.disabled = False
    previous_help_button.disabled = False

    # choose the start page of the beginner tour
    header_text_status = 1
    help_function()


def advanced_introduction_button_click(arg=None):
    pass


def show_subfigure_explenation_button_click(arg=None):
    pass


def show_parameter_explenation_button_click(arg=None):
    pass

actual help Panel

In [ ]:
help_intro_text = pn.pane.Markdown('Here you can jump to different sections of the help tour. The help will be shown ' +
                                   'in the Headerline and you can navigate through with the "next" and "previous" button. ',
                                   sizing_mode='stretch_width', height=20)

beginner_introduction_button = pn.widgets.Button(disabled=False,
                                                 name='beginner mode tour',
                                                 button_type='primary',
                                                 sizing_mode='stretch_width')
beginner_introduction_button.param.watch(
    beginner_introduction_button_click, 'clicks')

advanced_introduction_button = pn.widgets.Button(disabled=True,
                                                 name='advanced mode introduction',
                                                 button_type='primary',
                                                 sizing_mode='stretch_width')
advanced_introduction_button.param.watch(
    advanced_introduction_button_click, 'clicks')

show_subfigure_explenation_button = pn.widgets.Button(disabled=True,
                                                      name='show explenation of each subfigure',
                                                      button_type='primary',
                                                      sizing_mode='stretch_width')
show_subfigure_explenation_button.param.watch(
    show_subfigure_explenation_button_click, 'clicks')

show_parameter_explenation_button = pn.widgets.Button(disabled=True,
                                                      name='show explenation of parameters',
                                                      button_type='primary',
                                                      sizing_mode='stretch_width')
show_parameter_explenation_button.param.watch(
    show_parameter_explenation_button_click, 'clicks')

help_panel = pn.Row(help_intro_text,
                    pn.Column(beginner_introduction_button,
                              advanced_introduction_button),
                    pn.Column(show_subfigure_explenation_button,
                              show_parameter_explenation_button),
                    background=tab_background,
                    sizing_mode='stretch_width')

### Panel for plotting options

help functions

In [ ]:
show_color_count = 0


def show_velocity_changed(arg=None):
    global show_color_count
    set_glacier_height_curve()
    main_figure.event(button_name='show_velocity',
                      clicks=show_color_count)
    show_color_count += 1


def show_thickness_changed(arg=None):
    global show_color_count
    set_width_curve()
    main_figure.event(button_name='show_thickness',
                      clicks=show_color_count)
    show_color_count += 1

actual panel

!!! to work with the app in the notebook you must comment out the two lines: "show_velocity.param.watch(show_velocity_changed, 'value')" and "show_thickness.param.watch(show_thickness_changed, 'value')" !!!

In [ ]:
show_velocity = pn.widgets.Checkbox(name='show velocity as colors (top left figure)',
                                    sizing_mode='stretch_width')
show_velocity.param.watch(show_velocity_changed, 'value')

show_thickness = pn.widgets.Checkbox(name='show thickness as colors (bottom left figure)',
                                     sizing_mode='stretch_width')
show_thickness.param.watch(show_thickness_changed, 'value')

color_options = pn.pane.Markdown(('Here you can choose if the velocity or/and the thickness ' +
                                  'of the ice should be shown. The colorbar shows the intervals, ' +
                                  'maximum refers to the value shown in the "info text". ' +
                                  '(takes longer with colors)'),
                                 height=55,
                                 sizing_mode='stretch_width')


# is needed to directly get to bokeh styling options
def colorbar_hook(plot, element):
    plot.handles['plot'].border_fill_color = tab_background
    plot.handles['plot'].min_border_right = 50
    plot.handles['colorbar'].background_fill_color = tab_background
    plot.handles['colorbar'].height = 10


ColorBar = hv.Polygons({'x': [],
                        'y': [],
                        'value': [0]},
                       vdims='value').opts(hooks=[colorbar_hook],
                                           responsive=True,
                                           colorbar_position='top',
                                           colorbar_opts={'orientation': 'horizontal',
                                                          'major_label_overrides': {-1: '0',
                                                                                    -0.5: 'quater',
                                                                                    0: 'half',
                                                                                    0.5: 'three-quaters',
                                                                                    1: 'maximum'},
                                                          'major_label_text_font_size': '10pt'},
                                           yaxis=None,
                                           xaxis=None,
                                           toolbar=None,
                                           frame_height=0,
                                           cmap=colormap,
                                           colorbar=True)

plotting_option_panel = pn.Column(pn.Row(color_options,
                                         pn.Column(show_velocity,
                                                   show_thickness,
                                                   background=tab_background,
                                                   sizing_mode='stretch_width')),
                                  ColorBar,
                                  background=tab_background,
                                  sizing_mode='stretch_width')

### Panel for model options

help function for button click

In [ ]:
def set_model_options(arg=None):
    global dyears
    global max_calc_years

    dyears = dyears_model_slider.value
    max_calc_years = max_calc_years_slider.value
    current_model_options.object = current_model_options_text.format(
        dyears_model_slider.value, max_calc_years_slider.value)

actual panel

In [ ]:
dyears_model_slider = pn.widgets.IntSlider(name='show model progress every ... years',
                                           start=dyears_model_start,
                                           end=dyears_model_end,
                                           step=dyears_model_step,
                                           value=dyears_model_default,
                                           sizing_mode='stretch_width')

max_calc_years_slider = pn.widgets.IntSlider(name='maximum calculation year of model',
                                             start=max_calc_years_start,
                                             end=max_calc_years_end,
                                             step=max_calc_years_step,
                                             value=max_calc_years_default,
                                             sizing_mode='stretch_width')

current_model_options_text = 'Model currently showing progress every {} years\nand abort model after {} years of calculation'
current_model_options = pn.pane.Markdown(current_model_options_text.format(dyears_model_default, max_calc_years_default),
                                         sizing_mode='stretch_width')

set_model_options_button = pn.widgets.Button(name='set new model options',
                                             button_type='primary',
                                             sizing_mode='stretch_width')
set_model_options_button.param.watch(set_model_options, 'clicks')

model_option_panel = pn.Row(current_model_options,
                            dyears_model_slider,
                            max_calc_years_slider,
                            set_model_options_button,
                            background=tab_background,
                            sizing_mode='stretch_width')

### add all panels together as tabs

In [ ]:
tab_menu = pn.Tabs(('beginner mode', beginner_panel),
                   ('advanced mode', advanced_panel),
                   ('find help here (under construction)', help_panel),
                   ('plotting options', plotting_option_panel),
                   ('model options', model_option_panel),
                   background=tab_background)

## Define functions to set numeric values of parameters

### set glen a

In [ ]:
def set_glen_a():
    global glen_a

    if glens_creep_parameter.value == 'default':
        glen_a = cfg.PARAMS['glen_a']
    elif glens_creep_parameter.value == 'small':
        glen_a = cfg.PARAMS['glen_a'] / 10
    elif glens_creep_parameter.value == 'large':
        glen_a = cfg.PARAMS['glen_a'] * 10

### set fs

In [ ]:
def set_fs():
    global fs

    if sliding_parameter.value == 'no sliding':
        fs = 0
    elif sliding_parameter.value == 'sliding':
        fs = 5.7e-20

## Define function for creating curves

### set bed rock curves

In [ ]:
def set_bed_rock_curves():
    global bed_h
    global bed_rock_height_curve
    global widths
    global bed_rock_width_curve
    global ELA_x_position

    ELA = int(ELA_height.value)

    # look which bed rock profile was chosen
    if bed_rock_profile.value == 'linear':
        # look which slope was choosen and set the heigth at each point along the glacier
        if bed_rock_slope.value == 'steepest':
            bed_h = np.append(np.linspace(glacier_top_height, 0, np.round(nx / 4)),
                              np.zeros(nx - np.int(np.round(nx / 4))))
        elif bed_rock_slope.value == 'steeper':
            bed_h = np.append(np.linspace(glacier_top_height, 0, np.round(nx / 4) * 2),
                              np.zeros(nx - np.int(np.round(nx / 4) * 2)))
        elif bed_rock_slope.value == 'steep':
            bed_h = np.append(np.linspace(glacier_top_height, 0, np.round(nx / 4) * 3),
                              np.zeros(nx - np.int(np.round(nx / 4) * 3)))
        elif bed_rock_slope.value == 'flat':
            bed_h = np.linspace(glacier_top_height, 0, nx)
        elif bed_rock_slope.value == 'flatter':
            bed_h = np.linspace(glacier_top_height, 1500, nx)
        elif bed_rock_slope.value == 'flattest':
            bed_h = np.linspace(glacier_top_height, 3000, nx)

    elif bed_rock_profile.value == 'getting flatter':
        bed_h = np.array([(np.log(x / nx + 0.03) / np.log(0.5) * 0.2 + 0.03) *
                          glacier_top_height for x in np.arange(nx)])

    elif bed_rock_profile.value == 'getting steeper':
        bed_h = np.array([(np.log(-x / nx / 4 + 0.26) * 0.2 / np.log(2) + 1.3) *
                          glacier_top_height for x in np.arange(nx)])

    elif bed_rock_profile.value == 'cliff':
        # define extend of cliff
        cliff_top = 2500
        cliff_bottom = 2400

        bed_h = np.concatenate((np.linspace(glacier_top_height, cliff_top, nx/2),
                                np.linspace(cliff_bottom, glacier_bottom_height, nx/2)))

    # create curve of bed rock
    bed_rock_height_curve = hv.Area((distance_along_glacier, bed_h),
                                    'distance along glacier (km)', 'altitude (m)',
                                    ).opts(default_tools=default_tools,
                                           color='lightgray',
                                           line_alpha=0)

    # look which glacier width was selected
    if bed_rock_width.value == 'constant':
        widths = np.zeros(nx) + 4.

    elif bed_rock_width.value == 'getting wider':
        widths = np.array([(2 / nx * x + 0.5) * 2 for x in np.arange(nx)])

    elif bed_rock_width.value == 'getting narrower':
        widths = np.array([(-2 / nx * x + 2.5) * 2 for x in np.arange(nx)])

    elif bed_rock_width.value == 'wide at the top and narrow at the bottom':
        widths = np.append(np.ones(np.int(np.round(nx/2))) * 4,
                           np.ones(np.int(nx - np.round(nx/2))) * 2)

    # find index for ELA position on x-axis (distance along glacier)
    ELA_x_position = (np.abs(bed_h - ELA)).argmin()

    bed_rock_width_curve = hv.Area((distance_along_glacier, -widths/2, widths/2),
                                   kdims=['distance along glacier (km)'],
                                   vdims=['y', 'y2']
                                   # label='glacier rock bed'
                                   ).redim.label(x='distance along glacier (km)',
                                                 y='width (m)').opts(default_tools=default_tools,
                                                                     color='lightgray',
                                                                     line_alpha=0)

### set glacier height curve

In [ ]:
def set_glacier_height_curve():
    global model
    global run_surface_heights
    global glacier_height_curve

    ELA = int(ELA_height.value)

    if not np.array_equal(run_surface_heights[0], bed_h):
        glacier_height_curve = (hv.Area((distance_along_glacier, run_surface_heights[0]),
                                        'distance along glacier (km)', 'altitude (m)',
                                        # label='last'
                                        ).opts(default_tools=default_tools,
                                               line_dash='dashed',
                                               line_color='darkgray',
                                               color='white',
                                               alpha=0.6,
                                               line_alpha=1,
                                               line_width=3) *
                                hv.Area((distance_along_glacier, run_surface_heights[1]),
                                        'distance along glacier (km)', 'altitude (m)',
                                        label='glacier'
                                        ).opts(default_tools=default_tools,
                                               color='white',
                                               line_color='blue',
                                               alpha=0.6,
                                               line_alpha=1,
                                               line_width=3))
    else:
        glacier_height_curve = (hv.Area(([], []),
                                        'distance along glacier (km)', 'altitude (m)',
                                        # label='last'
                                        ).opts(default_tools=default_tools,
                                               line_dash='dashed',
                                               line_color='darkgray',
                                               color='white',
                                               alpha=0.6,
                                               line_alpha=1,
                                               line_width=3) *
                                hv.Area((distance_along_glacier, run_surface_heights[1]),
                                        'distance along glacier (km)', 'altitude (m)',
                                        label='glacier'
                                        ).opts(default_tools=default_tools,
                                               color='white',
                                               line_color='blue',
                                               alpha=1,
                                               line_alpha=1,
                                               line_width=3))

     # looking for potential segments which can be colored if wanted
    index_of_segments = np.arange(np.sum(model.fls[-1].thick > 0))

    # show ice velocity in colors if wanted and if there is something to show
    if show_velocity.value and index_of_segments.size != 0:
        # calculate the mean velocity between two segments
        plot_velocity = [(np.array(model.get_diagnostics()['ice_velocity'])[i] +
                          np.array(model.get_diagnostics()['ice_velocity'])[i + 1]) / 2
                         for i in index_of_segments]

        # is necessary so the line_color is the same as the filled color of the polygons
        # velocity_mapper = []
        velocity_mapper = linear_cmap(field_name='velocity', palette=colormap,
                                      low=np.min(plot_velocity),
                                      high=np.max(plot_velocity))

        # actual colored velocity
        glacier_height_curve *= hv.Polygons([{'x': [distance_along_glacier[i],
                                                    distance_along_glacier[i + 1],
                                                    distance_along_glacier[i + 1],
                                                    distance_along_glacier[i]],
                                              'y': [bed_h[i],  # 0,
                                                    bed_h[i+1],  # 0,
                                                    run_surface_heights[1][i + 1],
                                                    run_surface_heights[1][i]],
                                              'velocity': plot_velocity[i]}
                                             for i in index_of_segments],
                                            vdims='velocity').opts(default_tools=default_tools,
                                                                   line_color=velocity_mapper,
                                                                   cmap=colormap)
    else:
        # when there is no glacier or colors should not be shown
        # this is needed so bokeh knows about the Polygons element
        glacier_height_curve *= hv.Polygons([{'x': [],
                                              'y': [],
                                              'velocity': [0]}],
                                            vdims='velocity').opts(default_tools=default_tools)

    # add ELA to plot
    glacier_height_curve *= hv.Curve((distance_along_glacier, np.repeat(ELA, np.size(distance_along_glacier))),
                                     'distance along glacier (km)',
                                     'altitude (m)'
                                     ).opts(default_tools=default_tools,
                                            line_dash='dashed',
                                            line_width=3,
                                            color='black')

### set mb curve

In [ ]:
def set_mb_curve():
    global mb_model
    global mb_curve

    ELA = int(ELA_height.value)
    gradient = int(mb_gradient.value)

    mb_model = LinearMassBalance(ELA, grad=gradient)

    # define the range of the mass balance curve wich should be shown
    y_range_of_mb_curve = np.array([-20, 20])
    x_range_of_mb_curve = np.array([0, 4500])

    # get the annual mass balance for the given range of the x axis
    annual_mb = mb_model.get_annual_mb(x_range_of_mb_curve) * cfg.SEC_IN_YEAR

    mb_curve = (hv.Area((y_range_of_mb_curve, np.repeat(np.max(x_range_of_mb_curve), 2)),
                        'annual mass balance (m/yr)',
                        'altitude (m)',
                        label='mass gain'
                        ).opts(default_tools=default_tools,
                               color='lightgreen',
                               line_alpha=0) *
                hv.Area((y_range_of_mb_curve, np.repeat(ELA, 2)),
                        'annual mass balance (m/yr)',
                        'altitude (m)',
                        label='mass loss'
                        ).opts(default_tools=default_tools,
                               color='lightcoral',
                               line_alpha=0) *
                hv.Curve((annual_mb, x_range_of_mb_curve),
                         'annual mass balance (m/yr)',
                         'altitude (m)',
                         label='mass balance'
                         ).opts(default_tools=default_tools,
                                color='black') *
                hv.Curve((y_range_of_mb_curve, np.repeat(ELA, 2)),
                         'annual mass balance (m/yr)',
                         'altitude (m)',
                         label='ELA'
                         ).opts(default_tools=default_tools,
                                line_dash='dashed',
                                line_width=3,
                                color='black')
                )

### set width curve

In [ ]:
def set_width_curve():
    global width_curve
    global model
    global glacier_outlines

    # define lower and upper y limit which is shown
    y_min = -3  # in m
    y_max = 3

    if not np.array_equal(run_surface_heights[0], bed_h):
        width_curve = (hv.Area((distance_along_glacier, -glacier_outlines[0]/2, glacier_outlines[0]/2),
                               kdims=['distance along glacier (km)'],
                               vdims=['y', 'y2']
                               # label='old'
                               ).redim.label(x='distance along glacier (km)',
                                             y='width (m)').opts(default_tools=default_tools,
                                                                 line_dash='dashed',
                                                                 color='white',
                                                                 line_color='darkgray',
                                                                 alpha=0.6,
                                                                 line_alpha=1,
                                                                 line_width=3) *
                       hv.Area((distance_along_glacier, -glacier_outlines[1]/2, glacier_outlines[1]/2),
                               kdims=['distance along glacier (km)'],
                               vdims=['y', 'y2']
                               # label='new'
                               ).redim.label(x='distance along glacier (km)',
                                             y='width (m)').opts(default_tools=default_tools,
                                                                 color='white',
                                                                 line_color='blue',
                                                                 alpha=0.6,
                                                                 line_alpha=1,
                                                                 line_width=3))
    else:
        width_curve = (hv.Area(([], [], []),
                               kdims=['distance along glacier (km)'],
                               vdims=['y', 'y2']
                               # label='old'
                               ).redim.label(x='distance along glacier (km)',
                                             y='width (m)').opts(default_tools=default_tools,
                                                                 line_dash='dashed',
                                                                 color='white',
                                                                 line_color='darkgray',
                                                                 alpha=0.6,
                                                                 line_alpha=1,
                                                                 line_width=3) *
                       hv.Area((distance_along_glacier, -glacier_outlines[1]/2, glacier_outlines[1]/2),
                               kdims=['distance along glacier (km)'],
                               vdims=['y', 'y2']
                               # label='new'
                               ).redim.label(x='distance along glacier (km)',
                                             y='width (m)').opts(default_tools=default_tools,
                                                                 color='white',
                                                                 line_color='blue',
                                                                 alpha=1,
                                                                 line_alpha=1,
                                                                 line_width=3))

    # looking for potential segments which can be colored if wanted
    index_of_segments = np.arange(np.sum(model.fls[-1].thick > 0))

    # show ice velocity in colors if wanted and there is something to show
    if show_thickness.value and index_of_segments.size != 0:
        # calculate the mean thickness between two segments
        plot_thickness = [(np.array(model.get_diagnostics()['ice_thick'])[i] +
                           np.array(model.get_diagnostics()['ice_thick'])[i + 1]) / 2
                          for i in index_of_segments]

        # is necessary so the line_color is the same as the filled color of the polygons
        thickness_mapper = linear_cmap(field_name='thickness', palette=colormap,
                                       low=np.min(plot_thickness),
                                       high=np.max(plot_thickness))

        # actual colored thickness
        width_curve *= hv.Polygons([{'x': [distance_along_glacier[i],
                                           distance_along_glacier[i + 1],
                                           distance_along_glacier[i + 1],
                                           distance_along_glacier[i]],
                                     'y': [-glacier_outlines[1][i]/2,
                                           -glacier_outlines[1][i+1]/2,
                                           glacier_outlines[1][i+1]/2,
                                           glacier_outlines[1][i]/2],
                                     'thickness': plot_thickness[i]}
                                    for i in index_of_segments],
                                   vdims='thickness').opts(default_tools=default_tools,
                                                           line_color=thickness_mapper,
                                                           cmap=colormap)
    else:
        # when there is no glacier or colors should not be shown
        # this is needed so bokeh knows about the Polygons element
        width_curve *= hv.Polygons([{'x': [],
                                     'y': [],
                                     'thickness': [0]}],
                                   vdims='thickness').opts(default_tools=default_tools)

    width_curve *= (hv.Curve((distance_along_glacier, np.repeat(0, np.size(distance_along_glacier))),
                             'distance along glacier (km)',
                             'width (m)'  # ,
                             # label='center Flowline'
                             ).opts(default_tools=default_tools,
                                    color='black',
                                    line_width=1) *
                    hv.Curve((np.repeat(distance_along_glacier[ELA_x_position], 2), [y_min, y_max]),
                             'distance along glacier (km)',
                             'width (m)',
                             # label='ELA'
                             ).opts(default_tools=default_tools,
                                    line_dash='dashed',
                                    line_width=3,
                                    color='black'))

    width_curve.opts(ylim=(y_min, y_max))

### set info text

In [ ]:
def set_info_text(stage='new model'):
    global info_text
    global model

    if stage == 'new model':
        text = 'New Model \t geometry\ninitialised'
    else:
        if stage == 'running model':
            text = 'Model is running: '
        elif stage == 'equilibrium':
            text = 'Model reached equilibrium '
        elif stage == 'stopped model':
            text = 'Model finished '
        elif stage == 'aborted model':
            text = 'Model aborted (max calculation time reached, see options)'

        text += '\nTime: {:4.0f} years          Volume: '

        if model.volume_km3 > 1:
            text += '{:9.2f} km³\n'
            volume = model.volume_km3
        else:
            text += '{:9.0f} m³\n'
            volume = model.volume_m3

        text += 'max ice thickness: {:.0f} m          max ice velocity: {:.2e} m/s'
        text = text.format(model.yr,
                           volume,
                           np.max(model.get_diagnostics()['ice_thick']),
                           np.max(model.get_diagnostics()['ice_velocity']))

    info_text = hv.Text(0.5, 0.5, text).opts(default_tools=['save', 'reset'],
                                             show_frame=False,
                                             text_baseline='middle',
                                             text_align='center',
                                             text_font_size='10pt',
                                             text_font='courier',
                                             xaxis=None,
                                             yaxis=None)

## Define functions for intialize flowline and run model

### init flowline model

In [ ]:
def init_flowline_model():
    global model
    global bed_h
    global run_surface_heights
    global widths
    global glacier_outlines

    # set info text
    set_info_text(stage='new model')

    # initialize the bed rock
    set_bed_rock_curves()

    # set glacier surface height to bed height because at the beginning there is no glacier
    surface_h = bed_h

    # initialize flowline with rectangular shape of glacier
    model_flowline = RectangularBedFlowline(surface_h=surface_h, bed_h=bed_h,
                                            widths=widths, map_dx=map_dx)

    model = FlowlineModel(model_flowline, mb_model=mb_model, y0=0., glen_a=glen_a,
                          fs=fs, check_for_boundaries=False)

    # set surface height for plotting, at beginn old and new one are the same
    run_surface_heights = np.repeat(
        np.array(model.fls[-1].surface_h, ndmin=2), 2, axis=0)

    # set glacier outlines for plotting, at beginn old and new one are the same
    glacier_outlines = np.repeat(np.array(
        np.where(model.fls[-1].thick > 0, widths, nan_array), ndmin=2), 2, axis=0)

### run the model

In [ ]:
def run_the_model(stage='beginner mode'):
    global model
    global run_surface_heights
    global run_loop_count
    global glacier_outlines
    global dyears
    global max_calc_years

    # set new mass balance curve
    set_mb_curve()

    if stage == 'beginner mode':
        # initialize a new model
        init_flowline_model()

    elif stage == 'advance equilibrium' or stage == 'advance years':
        # initialize model with old extend and new mass balance model
        model = FlowlineModel(model.fls[-1], mb_model=mb_model, y0=0.,
                              glen_a=glen_a, fs=fs, check_for_boundaries=False)

        # determine glacier state at starting point
        run_surface_heights = np.repeat(
            np.array(model.fls[-1].surface_h, ndmin=2), 2, axis=0)
        glacier_outlines = np.repeat(np.array(
            np.where(model.fls[-1].thick > 0, widths, nan_array), ndmin=2), 2, axis=0)

    if stage == 'beginner mode':
        set_glacier_height_curve()
        set_width_curve()

    if stage == 'beginner mode' or stage == 'advance equilibrium':
        # mostly same function as from OGGM FlowlineModel.run_until_equilibrium
        was_close_zero = 0
        t_rate = 1
        while (t_rate > 0.001) and (model.yr < max_calc_years) and (was_close_zero < 5):
            v_bef = model.volume_m3
            model.run_until(model.yr + dyears)
            v_af = model.volume_m3
            if np.isclose(v_bef, 0., atol=1):
                t_rate = 1
                was_close_zero += 1
            else:
                t_rate = np.abs(v_af - v_bef) / v_bef

            # set text with current year, volume, thickness and velocity
            set_info_text(stage='running model')

            # save new surface/outline and show it on plot
            run_surface_heights[1] = np.array(model.fls[-1].surface_h, ndmin=2)
            glacier_outlines[1] = np.array(
                np.where(model.fls[-1].thick > 0, widths, nan_array), ndmin=2)
            set_glacier_height_curve()
            set_width_curve()
            main_figure.event(button_name='run_button',
                              clicks=run_loop_count)
            run_loop_count += 1

        if model.yr >= max_calc_years:
            set_info_text(stage='aborted model')
        else:
            # set text with equilibrium year
            set_info_text(stage='equilibrium')

        main_figure.event(button_name='run_button',
                          clicks=run_loop_count)

    elif stage == 'advance years':
        while (model.yr < run_model_years.value):
            model.run_until(model.yr + dyears)

            # set text with current year and volume
            set_info_text(stage='running model')

            # save new surface/outline and show it on plot
            run_surface_heights[1] = np.array(model.fls[-1].surface_h, ndmin=2)
            glacier_outlines[1] = np.array(
                np.where(model.fls[-1].thick > 0, widths, nan_array), ndmin=2)
            set_glacier_height_curve()
            set_width_curve()
            main_figure.event(button_name='run_button',
                              clicks=run_loop_count)
            run_loop_count += 1

        # set text after calculation
        set_info_text(stage='stopped model')
        main_figure.event(button_name='run_button',
                          clicks=run_loop_count)

## Main function for dynamic map

In [ ]:
def change_plot(button_name='new', clicks=0):
    # use global curves
    global bed_rock_height_curve
    global bed_rock_width_curve
    global glacier_height_curve
    global mb_curve
    global width_curve

    if button_name == 'new':
        # TODO: set all widgets to default values
        # bed_rock_profile.value = 'linear'

        # initialisation of figure
        set_mb_curve()
        init_flowline_model()
        set_glacier_height_curve()
        set_width_curve()

    elif button_name == 'new model':
        set_mb_curve()
        init_flowline_model()
        set_glacier_height_curve()
        set_width_curve()


    return ((glacier_height_curve * bed_rock_height_curve).opts(legend_position='bottom_left',
                                                                legend_cols=2,
                                                                xaxis='top',
                                                                bgcolor='lightblue',
                                                                responsive=True,
                                                                aspect=2.8) +
            (mb_curve).opts(legend_position='bottom_right',
                            legend_cols=4,
                            xaxis='top',
                            yaxis='right',
                            responsive=True,
                            aspect=2.8) +
            (bed_rock_width_curve * width_curve).opts(responsive=True,
                                                      aspect=6,
                                                      yformatter='%8.0f',
                                                      bgcolor='darkgray',
                                                      labelled=['y']) +
            (info_text).opts(responsive=True,
                             aspect=6)
            ).opts(sizing_mode='stretch_both',
                   toolbar='right').cols(2)

## streamer and dynamic map

In [ ]:
main_stream = Stream.define('main_stream', button_name='new', clicks=0)

main_figure = hv.DynamicMap(change_plot, streams=[main_stream()])

## App

In [ ]:
pn.Column(header, tab_menu, main_figure, sizing_mode='stretch_both').servable()

As long as you are running this notebook "live" (in Jupyter, not viewing a website or a static copy), the above notebook cell should contain the fully operational dashboard here in the notebook. You can also launch the dashboard at a separate port that shows up in a new browser tab, either by changing .servable() to .show() above and re-executing that cell, or by leaving the cell as it is and running bokeh serve --show simulator.ipynb.

!!! Changing the colors when clicking the checkbox only operates when the app is executed with .show(). If you want work in the notebook without an error go to the section "panel for plotting options" and see description. (for more information you can have a look at <a href=https://github.com/bokeh/bokeh/issues/8579>https://github.com/bokeh/bokeh/issues/8579</a>) !!!